# Graph Coloring Problem

The **graph coloring problem** can be described mathematically as follows:

Let G = (V, E) be an undirected graph, where:

- V is the set of vertices (or nodes) of the graph.
- E is the set of edges, where each edge e = (u, v) represents a connection between two vertices u and v.

---

### Objective

The goal is to assign a color to each vertex in V such that no two adjacent vertices share the same color. The **chromatic number** χ(G) of the graph G is the minimum number of colors required to color the vertices of the graph in this way. Formally:

χ(G) = min { k : there exists a function f : V → {1, 2, ..., k} such that for all (u, v) ∈ E, f(u) ≠ f(v) }

Where:

- f : V → {1, 2, ..., k} is a coloring function that assigns a color (represented by a number) to each vertex.
- The constraint f(u) ≠ f(v) ensures that adjacent vertices do not share the same color.

---

### Computational Complexity

Determining the chromatic number χ(G) of a graph is a well-known **NP-hard** problem. This means that, in general, there is no efficient algorithm that can find the exact chromatic number for arbitrary graphs in polynomial time, unless P = NP.

Thus, finding the optimal solution (i.e., the exact chromatic number) is computationally expensive, especially for large graphs. This makes exact solutions impractical for real-world applications where graphs can have thousands or millions of vertices and edges.

---

### Heuristic Solutions

Given the difficulty of solving the graph coloring problem exactly, **heuristics** are commonly used to find approximate solutions. These heuristics aim to find a solution that is close to the optimal chromatic number, but in a much faster time frame.

- **DSatur (Degree of Saturation)**: This heuristic prioritizes vertices based on the number of differently colored neighbors (saturation degree). The vertex with the highest saturation degree is colored first, as it has the least flexibility in terms of color assignment. The algorithm iteratively selects the vertex with the highest saturation degree and assigns it the lowest color that is not used by its neighbors.

- **Welsh-Powell**: This heuristic colors vertices in decreasing order of their degrees. The idea is that vertices with a high degree (more neighbors) are more constrained in terms of color choices and should be colored first.

- **RLF (Recursive Largest First)**: This heuristic recursively selects the largest independent set of uncolored vertices, colors them, and then repeats the process with the remaining vertices. It prioritizes vertices based on their degree and the degree of their neighbors, aiming to minimize conflicts.

In [31]:
import time
from abc import ABC, abstractmethod
from collections import defaultdict
from heapq import heappush, heappop

### Undirected Graph Representation Using an Adjacency List

The `Graph` class models a simple undirected graph using an **adjacency list** representation. This approach is well-suited for **sparse graphs**, where the number of edges is significantly lower than the total number of possible connections.

#### Key characteristics:
- The graph is **undirected**, meaning edges are bidirectional.
- Nodes are indexed as integers from `0` to `n-1`.
- Each node maintains a list of its directly connected neighbors.

#### Main methods:
- `add_edge(u, v)`: Adds an undirected edge between nodes `u` and `v`.
- `get_neighbors(v)`: Returns the list of nodes adjacent to node `v`.
- `get_degree(v)`: Returns the number of neighbors (degree) of node `v`.

In [32]:
class Graph:
    """Représente un graphe simple non-orienté."""
    def __init__(self, num_nodes):
        self.n = num_nodes
        self.adj = [[] for _ in range(num_nodes)]  # Liste d'adjacence

    def add_edge(self, u, v):
        """Ajoute une arête entre u et v."""
        self.adj[u].append(v)
        self.adj[v].append(u)

    def get_neighbors(self, v):
        """Retourne les voisins du sommet v."""
        return self.adj[v]

    def get_degree(self, v):
        """Retourne le degré du sommet v."""
        return len(self.adj[v])

## Abstract Interface for Graph Coloring Algorithms

The `ColoringAlgorithm` class defines an **abstract base class (ABC)** for implementing graph coloring strategies. It provides a common structure and reusable utilities for any specific coloring algorithm, such as greedy coloring, DSATUR, or metaheuristic-based methods.

#### Purpose:
This interface enforces a standard method signature for coloring algorithms, ensuring that all subclasses implement a `color_graph()` method. It also provides basic utilities for color assignment and result reporting.

#### Attributes:
- `graph`: The input graph on which coloring is to be applied.
- `colors`: A list where each index corresponds to a node and the value is its assigned color. Initialized to `-1` for all nodes, meaning uncolored.

#### Main methods:
- `color_graph()`: Abstract method to be implemented by subclasses. It contains the core logic for coloring the graph.
- `get_colors()`: Returns the list of assigned colors for all nodes.
- `display_results(execution_time=None)`: Prints the color assigned to each node, the total number of colors used, and optionally the execution time.



In [33]:
class ColoringAlgorithm(ABC):
    """Classe abstraite définissant l'interface d'un algorithme de coloration."""

    def __init__(self, graph):
        self.graph = graph
        self.colors = [-1] * graph.n  # Tableau des couleurs

    @abstractmethod
    def color_graph(self):
        """Applique l'algorithme de coloration au graphe."""
        pass

    def get_colors(self):
        """Retourne le tableau des couleurs."""
        return self.colors

    def display_results(self, execution_time=None):
        """Affiche les résultats de coloration."""
        for v in range(self.graph.n):
            print(f"Sommet {v} ---> Couleur {self.colors[v]}")

        nb_colors = max(self.colors) + 1
        print(f"\nNombre total de couleurs utilisées : {nb_colors}")

        if execution_time is not None:
            print(f"Temps d'exécution : {execution_time:.4f} ms")

        return nb_colors

## Abstract Interface for Vertex Selection Strategies

The `VertexSelector` class defines an **abstract base interface** for implementing vertex selection strategies used in graph coloring algorithms. These strategies determine the order in which vertices are selected for coloring, which can significantly influence the quality and efficiency of the final solution.

#### Purpose:
This interface allows for the implementation of various selection heuristics, such as:
- Degree-based ordering
- DSATUR (Degree of Saturation)
- Random selection
- Hybrid or adaptive approaches

#### Main methods:
- `initialize(graph)`: Prepares internal data structures using the provided graph. Typically called once before coloring begins.
- `get_next_vertex()`: Returns the next vertex to be colored, based on the current state and selection strategy.
- `update_vertex(vertex, color, colored_vertices)`: Updates priorities or internal tracking after a vertex has been assigned a color.
- `is_finished()`: Checks whether all vertices have been processed.

By decoupling vertex selection from the coloring logic, this design supports modular experimentation and comparison of different heuristics.

In [34]:
class VertexSelector(ABC):
    """Classe abstraite définissant la sélection de sommet prioritaire."""

    @abstractmethod
    def initialize(self, graph):
        """Initialise la structure de données avec le graphe donné."""
        pass

    @abstractmethod
    def get_next_vertex(self):
        """Retourne le prochain sommet à colorer selon la stratégie choisie."""
        pass

    @abstractmethod
    def update_vertex(self, vertex, color, colored_vertices):
        """Met à jour la priorité des sommets après coloration d'un sommet."""
        pass

    @abstractmethod
    def is_finished(self):
        """Vérifie si tous les sommets ont été traités."""
        pass

## DSatur - Degree Of Saturation

### DSatur-Based Vertex Selector Using a Priority Queue (Heap)

The `DSaturHeapSelector` class implements a vertex selection strategy based on the **DSATUR (Degree of Saturation)** heuristic, enhanced with a **priority queue (heap)** to efficiently select the next vertex to color.

#### Overview:
DSATUR is a popular heuristic in graph coloring where the vertex with the highest **saturation degree** (number of different colors used by its neighbors) is selected next. In case of a tie, the vertex with the highest **degree** is preferred.

This implementation uses a **max-heap** to maintain selection priorities efficiently.

#### Attributes:
- `graph`: The input graph.
- `pq`: A max-heap containing tuples of the form `(-saturation, -degree, vertex)` to prioritize vertices.
- `remaining`: A set of uncolored vertices.
- `saturation`: An array tracking the number of distinct neighbor colors for each vertex.
- `adj_colors`: A list of sets storing the colors used by each vertex's neighbors.

#### Method behavior:
- `initialize(graph)`: Sets up the internal data structures and fills the priority queue with all vertices using their degree.
- `get_next_vertex()`: Pops the most prioritized vertex from the heap that is still uncolored.
- `update_vertex(vertex, color, colored_vertices)`: Updates saturation levels of the vertex’s neighbors and re-inserts them into the heap with new priorities.
- `is_finished()`: Checks whether all vertices have been colored.


In [35]:
class DSaturHeapSelector(VertexSelector):
    """Sélecteur de sommet utilisant une file de priorité (heap)."""

    def __init__(self):
        self.graph = None
        self.pq = []
        self.remaining = set()
        self.saturation = []
        self.adj_colors = []

    def initialize(self, graph):
        self.graph = graph
        self.remaining = set(range(graph.n))
        self.saturation = [0] * graph.n
        self.adj_colors = [set() for _ in range(graph.n)]
        self.pq = []

        # Initialiser la file de priorité avec tous les sommets
        for v in range(graph.n):
            degree = graph.get_degree(v)
            heappush(self.pq, (-0, -degree, v))  # Négation pour max-heap

    def get_next_vertex(self):
        # Récupérer le sommet le plus prioritaire
        while self.pq and self.remaining:
            _, _, v = heappop(self.pq)
            if v in self.remaining:
                return v
        return None

    def update_vertex(self, vertex, color, colored_vertices):
        # Mettre à jour les voisins du sommet colorié
        self.remaining.remove(vertex)

        for neighbor in self.graph.get_neighbors(vertex):
            if neighbor in self.remaining:
                if color not in self.adj_colors[neighbor]:
                    self.adj_colors[neighbor].add(color)
                    self.saturation[neighbor] += 1

                # Ajouter le voisin avec sa priorité mise à jour
                degree = self.graph.get_degree(neighbor)
                heappush(self.pq, (-self.saturation[neighbor], -degree, neighbor))

    def is_finished(self):
        return len(self.remaining) == 0

### DSatur-Based Vertex Selector Using Bucket Structures

The `DSaturBucketSelector` class implements the **DSATUR vertex selection heuristic** using a **bucket-based data structure** instead of a priority queue. This method can offer improved performance in certain cases by grouping vertices with identical priorities into discrete sets (buckets), reducing the need for heap reordering.

#### Core idea:
Each vertex is stored in a bucket identified by a composite key: `(saturation degree, vertex degree, vertex index)`. At each step, the vertex from the **maximum bucket** is selected, following the DSATUR logic:
1. Maximize saturation (number of unique neighbor colors),
2. Break ties by choosing the vertex with the highest degree.

#### Attributes:
- `graph`: The input graph.
- `buckets`: A dictionary mapping priority keys to sets of vertices sharing those priorities.
- `saturation`: Tracks the number of distinct neighbor colors for each vertex.
- `adj_colors`: Maintains the set of colors used by each vertex's neighbors.

#### Method behavior:
- `initialize(graph)`: Initializes all vertices with saturation 0 and places them in corresponding buckets based on degree.
- `get_next_vertex()`: Finds the vertex in the bucket with the highest key and removes it from the structure.
- `update_vertex(vertex, color, colored_vertices)`: After coloring a vertex, updates its neighbors' saturation and moves them to the appropriate new bucket.
- `is_finished()`: Returns `True` when all buckets are empty, meaning all vertices have been colored.

In [36]:
class DSaturBucketSelector(VertexSelector):
    """Sélecteur de sommet utilisant une structure de buckets."""

    def __init__(self):
        self.graph = None
        self.buckets = defaultdict(set)
        self.saturation = []
        self.adj_colors = []

    def initialize(self, graph):
        self.graph = graph
        self.saturation = [0] * graph.n
        self.adj_colors = [set() for _ in range(graph.n)]
        self.buckets = defaultdict(set)

        # Initialiser les buckets avec tous les sommets
        for v in range(graph.n):
            degree = graph.get_degree(v)
            self.buckets[(0, degree, v)].add(v)

    def get_next_vertex(self):
        if not self.buckets:
            return None

        # Trouver la clé maximale dans les buckets
        max_key = max(self.buckets.keys())

        # Obtenir et supprimer un sommet de ce bucket
        v = next(iter(self.buckets[max_key]))
        self.buckets[max_key].remove(v)
        if not self.buckets[max_key]:
            del self.buckets[max_key]

        return v

    def update_vertex(self, vertex, color, colored_vertices):
        # Mettre à jour les voisins du sommet colorié
        for neighbor in self.graph.get_neighbors(vertex):
            if neighbor not in colored_vertices:
                # Supprimer le voisin de son bucket actuel
                old_key = (self.saturation[neighbor], self.graph.get_degree(neighbor), neighbor)
                if neighbor in self.buckets[old_key]:
                    self.buckets[old_key].remove(neighbor)
                    if not self.buckets[old_key]:
                        del self.buckets[old_key]

                # Mettre à jour la saturation
                if color not in self.adj_colors[neighbor]:
                    self.adj_colors[neighbor].add(color)
                    self.saturation[neighbor] += 1

                # Ajouter le voisin dans son nouveau bucket
                new_key = (self.saturation[neighbor], self.graph.get_degree(neighbor), neighbor)
                self.buckets[new_key].add(neighbor)

    def is_finished(self):
        return len(self.buckets) == 0

### Greedy Color Assignment Utility

The `ColorAssigner` class provides a utility for assigning colors to graph vertices using a **greedy strategy**. It operates independently of the vertex selection logic and focuses solely on determining the **smallest valid color** that can be assigned to each vertex.

#### Purpose:
This class is typically used in conjunction with vertex selectors (e.g., DSATUR) to perform the actual coloring step. It ensures that no adjacent vertices share the same color, maintaining the validity of the solution.

#### Attributes:
- `graph`: The input graph.
- `colors`: A list storing the color assigned to each vertex, initialized to `-1` (uncolored).
- `adj_colors`: A list of sets storing the colors currently used by each vertex’s neighbors.

#### Main methods:
- `assign_color(vertex)`: Finds and assigns the **smallest available color** not used by any of the vertex’s neighbors. Returns the assigned color.
- `update_adj_colors(vertex, color)`: After a vertex is colored, updates the neighbor color sets to reflect this change.
- `get_colors()`: Returns the list of all assigned vertex colors.

In [37]:
class ColorAssigner:
    """Responsable de l'assignation des couleurs aux sommets."""

    def __init__(self, graph):
        self.graph = graph
        self.colors = [-1] * graph.n
        self.adj_colors = [set() for _ in range(graph.n)]

    def assign_color(self, vertex):
        """Assigne la plus petite couleur disponible au sommet."""
        used_colors = self.adj_colors[vertex]

        # Trouver la plus petite couleur non utilisée par les voisins
        for color in range(self.graph.n):
            if color not in used_colors:
                self.colors[vertex] = color
                return color

        return -1  # En cas d'erreur

    def update_adj_colors(self, vertex, color):
        """Met à jour les couleurs adjacentes aux voisins."""
        for neighbor in self.graph.get_neighbors(vertex):
            if self.colors[neighbor] == -1:  # Si le voisin n'est pas encore colorié
                self.adj_colors[neighbor].add(color)

    def get_colors(self):
        """Retourne le tableau des couleurs."""
        return self.colors

### DSATUR Graph Coloring Algorithm

The `DSaturColoring` class implements the **DSATUR (Degree of Saturation)** graph coloring algorithm. DSATUR is a heuristic approach that selects the next vertex to color based on its **saturation degree** (the number of different colors used by its neighbors), with ties broken by vertex degree (the number of edges connected to the vertex).

#### Overview:
This implementation allows users to choose between two vertex selection strategies:
- **Heap-based DSATUR** (using a priority queue), or
- **Bucket-based DSATUR** (using a bucket structure for grouping vertices with the same priority).

The algorithm colors the graph by:
1. Selecting the vertex with the highest saturation (and, in case of ties, the highest degree).
2. Assigning it the smallest available color that doesn’t conflict with its neighbors.
3. Updating the saturation of its neighbors.

#### Attributes:
- `selector`: The vertex selection strategy (either heap or bucket).
- `color_assigner`: Responsible for assigning colors to vertices.
- `colored_vertices`: A set tracking which vertices have been assigned colors.

#### Main methods:
- `color_graph()`: Applies the DSATUR algorithm to color the graph, measuring execution time and returning the color assignments and the total time taken.

In [38]:
class DSaturColoring(ColoringAlgorithm):
    """Algorithme principal de coloration DSatur."""

    def __init__(self, graph, selector_type="bucket"):
        """
        Initialise l'algorithme DSatur.

        Args:
            graph: Le graphe à colorer
            selector_type: Type de sélecteur ("heap" ou "bucket")
        """
        super().__init__(graph)

        # Choisir le sélecteur approprié
        if selector_type == "heap":
            self.selector = DSaturHeapSelector()
        else:
            self.selector = DSaturBucketSelector()

        self.color_assigner = ColorAssigner(graph)
        self.colored_vertices = set()

    def color_graph(self):
        """Applique l'algorithme DSatur pour colorer le graphe."""
        start_time = time.time()

        # Initialiser le sélecteur
        self.selector.initialize(self.graph)

        # Colorer tous les sommets un par un
        while not self.selector.is_finished():
            # Sélectionner le prochain sommet à colorer
            vertex = self.selector.get_next_vertex()
            if vertex is None:
                break

            # Assigner une couleur au sommet
            color = self.color_assigner.assign_color(vertex)

            # Mettre à jour les couleurs adjacentes
            self.color_assigner.update_adj_colors(vertex, color)

            # Mettre à jour le sélecteur
            self.colored_vertices.add(vertex)
            self.selector.update_vertex(vertex, color, self.colored_vertices)

        # Mesurer le temps d'exécution
        execution_time = (time.time() - start_time) * 1000

        # Récupérer les résultats
        self.colors = self.color_assigner.get_colors()

        return self.colors, execution_time

# Welsh-Powell Graph Coloring Algorithm

The `WelshPowellColoring` class implements the **Welsh-Powell** graph coloring algorithm. This greedy algorithm focuses on coloring vertices in decreasing order of their degree. The general approach is to assign the same color to all vertices that do not share an edge, thus forming independent sets that are colored together.

#### Overview:
The algorithm works by:
1. Sorting the vertices by **degree** in descending order.
2. Iteratively assigning the current available color to an **independent set** of vertices (i.e., vertices that are not adjacent to each other).
3. Repeating this process until all vertices are colored.

This approach is **greedy**, and while it does not guarantee the optimal number of colors, it often produces good results in practice.

#### Main methods:
- `color_graph()`: Executes the Welsh-Powell algorithm to color the graph. It measures the total execution time and returns the final color assignments for all vertices.

#### Characteristics:
- The algorithm is **greedy** and runs in **O(n^2)** time complexity, where `n` is the number of vertices, since it requires sorting vertices and checking for adjacency within independent sets.
- It is simple to implement and can provide a quick, though possibly suboptimal, solution to graph coloring problems.



In [39]:
class WelshPowellColoring(ColoringAlgorithm):
    """Implémentation de l'algorithme de Welsh-Powell."""

    def color_graph(self):
        """Applique l'algorithme Welsh-Powell pour colorer le graphe."""
        start_time = time.time()

        # Trier les sommets par degré décroissant
        vertices = sorted(range(self.graph.n), key=lambda v: self.graph.get_degree(v), reverse=True)

        color = 0
        uncolored = set(vertices)

        while uncolored:
            # Trouver un ensemble indépendant
            independent_set = set()

            for v in vertices:
                if v in uncolored:
                    # Vérifier si le sommet peut recevoir la couleur actuelle
                    can_color = True
                    for u in independent_set:
                        if v in self.graph.get_neighbors(u):
                            can_color = False
                            break

                    if can_color:
                        independent_set.add(v)
                        self.colors[v] = color

            # Retirer les sommets coloriés
            uncolored -= independent_set

            # Passer à la couleur suivante
            color += 1

        execution_time = (time.time() - start_time) * 1000

        return self.colors, execution_time

# Recursive Largest First (RLF) Graph Coloring Algorithm

The `RecursiveLargestFirstColoring` class implements the **Recursive Largest First (RLF)** algorithm for graph coloring. RLF is a heuristic coloring algorithm that selects the largest independent sets recursively and colors them with the smallest available color.

#### Overview:
The RLF algorithm works as follows:
1. Iteratively selects the vertex with the **maximum degree** among the uncolored vertices.
2. Forms an **independent set** by selecting vertices that are not adjacent to the current vertex.
3. Chooses the vertex that minimizes the number of edges to other uncolored vertices, adding it to the independent set.
4. Colors all vertices in the independent set with the current color and proceeds to the next color.

This algorithm is known for its efficiency and often results in fewer colors than simpler greedy algorithms, although it does not always guarantee an optimal solution.

#### Main methods:
- `color_graph()`: Executes the RLF algorithm to color the graph. The method tracks the execution time and returns the final color assignments for all vertices.

#### Characteristics:
- The algorithm is **recursive**, selecting independent sets one by one.
- It operates in **O(n^2)** time complexity, where `n` is the number of vertices, as it requires scanning through neighbors and calculating the independent set.
- While it often provides better results than basic greedy approaches, it does not guarantee an optimal solution.

In [40]:
class RecursiveLargestFirstColoring(ColoringAlgorithm):
    """Implémentation de l'algorithme Recursive Largest First (RLF)."""

    def color_graph(self):
        """Applique l'algorithme RLF pour colorer le graphe."""
        start_time = time.time()

        uncolored = set(range(self.graph.n))
        color = 0

        while uncolored:
            # Trouver le sommet de degré maximum
            v = max(uncolored, key=lambda x: len([n for n in self.graph.get_neighbors(x) if n in uncolored]))

            # Initialiser l'ensemble indépendant avec ce sommet
            independent_set = {v}

            # Sélectionner les candidats (sommets non adjacents à v)
            candidates = uncolored - {v} - set(self.graph.get_neighbors(v))

            while candidates:
                # Choisir le sommet qui minimise le nombre de connexions avec d'autres candidats
                w = min(candidates, key=lambda x: sum(1 for n in self.graph.get_neighbors(x) if n in candidates))

                # Ajouter ce sommet à l'ensemble indépendant
                independent_set.add(w)

                # Mettre à jour les candidats (retirer w et ses voisins)
                candidates.remove(w)
                candidates -= set(self.graph.get_neighbors(w))

            # Colorer l'ensemble indépendant
            for node in independent_set:
                self.colors[node] = color
                uncolored.remove(node)

            # Passer à la couleur suivante
            color += 1

        execution_time = (time.time() - start_time) * 1000

        return self.colors, execution_time

In [41]:
def read_dimacs_graph(file_path):
    """Lit un graphe au format DIMACS à partir d'un fichier."""
    with open(file_path, 'r') as file:
        lines = file.readlines()

    num_nodes = 0
    edges = []

    for line in lines:
        if line.startswith('p'):  # Ligne contenant le nombre de sommets
            parts = line.split()
            num_nodes = int(parts[2])
        elif line.startswith('e'):  # Ligne contenant une arête
            parts = line.split()
            node1, node2 = int(parts[1]), int(parts[2])
            edges.append((node1 - 1, node2 - 1))  # Convertir en index 0-based

    # Création du graphe
    graph = Graph(num_nodes)
    for u, v in edges:
        graph.add_edge(u, v)

    return graph

In [42]:
def compare_algorithms(graph):
    """Compare différents algorithmes de coloration sur un même graphe."""
    algorithms = [
        ("DSatur (Bucket)", DSaturColoring(graph, "bucket")),
        ("DSatur (Heap)", DSaturColoring(graph, "heap")),
        ("Welsh-Powell", WelshPowellColoring(graph)),
        ("RLF", RecursiveLargestFirstColoring(graph))
    ]

    results = []

    for name, algo in algorithms:
        print(f"\n=== {name} ===")
        colors, execution_time = algo.color_graph()
        nb_colors = algo.display_results(execution_time)
        results.append((name, nb_colors, execution_time))
def read_dimacs_graph(file_path):
    """Lit un graphe au format DIMACS à partir d'un fichier."""
    with open(file_path, 'r') as file:
        lines = file.readlines()

    num_nodes = 0
    edges = []

    for line in lines:
        if line.startswith('p'):  # Ligne contenant le nombre de sommets
            parts = line.split()
            num_nodes = int(parts[2])
        elif line.startswith('e'):  # Ligne contenant une arête
            parts = line.split()
            node1, node2 = int(parts[1]), int(parts[2])
            edges.append((node1 - 1, node2 - 1))  # Convertir en index 0-based

    # Création du graphe
    graph = Graph(num_nodes)
    for u, v in edges:
        graph.add_edge(u, v)

    return graph
    print("\n=== Résumé comparatif ===")
    print("Algorithme\t\tNombre de couleurs\tTemps (ms)")
    print("------------------------------------------------")
    for name, nb_colors, time in results:
        print(f"{name:<20}\t{nb_colors}\t\t{time:.4f}")

In [43]:
def test_coloring_algorithms(graph_files, time_limit=300):
    """
    Test different graph coloring algorithms on a list of graph files.

    Args:
        graph_files: Dictionary {graph_name: file_path}
        time_limit: Time limit in seconds for each execution (not strictly enforced)

    Returns:
        DataFrame with the results
    """
    import pandas as pd
    import time
    import signal
    from collections import defaultdict

    # Define known best K for certain graphs (optimal or best known)
    known_best_k = {
        'dsjc250.5': 28,
        'dsjc500.9': 126,
        'dsjr500.1c': 84,
        'flat1000_76_0': 76,
        'r250.5': 65
    }

    # Define algorithms to test
    algorithms = [
        "DSatur (Bucket)",
        "DSatur (Heap)",
        "Welsh-Powell",
        "RLF"
    ]

    # Prepare DataFrame for results
    results = []

    # For each graph
    for graph_name, graph_file in graph_files.items():
        print(f"\nProcessing graph: {graph_name}")

        # Load the graph
        try:
            graph = read_dimacs_graph(graph_file)
            print(f"Graph loaded: {graph.n} vertices")
        except Exception as e:
            print(f"Error loading graph {graph_name}: {e}")
            continue

        # For each algorithm
        for algo_name in algorithms:
            print(f"Testing algorithm: {algo_name}")

            # Create the algorithm instance
            if algo_name == "DSatur (Bucket)":
                algorithm = DSaturColoring(graph, "bucket")
            elif algo_name == "DSatur (Heap)":
                algorithm = DSaturColoring(graph, "heap")
            elif algo_name == "Welsh-Powell":
                algorithm = WelshPowellColoring(graph)
            elif algo_name == "RLF":
                algorithm = RecursiveLargestFirstColoring(graph)
            else:
                continue

            # Execute the algorithm
            start_time = time.time()
            try:
                colors, execution_time_ms = algorithm.color_graph()
                k = max(colors) + 1  # Number of colors used

                # Check for conflicts (validate coloring)
                conflicts = 0
                for u in range(graph.n):
                    for v in graph.get_neighbors(u):
                        if colors[u] == colors[v]:
                            conflicts += 1
                # Each edge is counted twice (once from each end), so divide by 2
                conflicts //= 2

            except Exception as e:
                print(f"Error executing {algo_name} on {graph_name}: {e}")
                k = float('inf')
                conflicts = float('inf')
                execution_time_ms = float('inf')

            execution_time = execution_time_ms / 1000  # Convert to seconds

            # Calculate gap from known best K
            known_best = known_best_k.get(graph_name, float('inf'))
            gap = ((k - known_best) / known_best * 100) if known_best < float('inf') and k < float('inf') else float('inf')

            # Analyze color usage
            color_usage = defaultdict(int)
            for color in colors:
                color_usage[color] += 1

            color_distribution = {color: count for color, count in sorted(color_usage.items())}

            # Add results
            result = {
                'Graph': graph_name,
                'Algorithm': algo_name,
                'K': k,
                'Known Best K': known_best if known_best < float('inf') else 'Unknown',
                'Gap (%)': f"{gap:.2f}" if gap != float('inf') else 'N/A',
                'Conflicts': conflicts,
                'Time (s)': execution_time,
                'Valid Coloring': conflicts == 0,
                'Color Distribution': str(color_distribution)
            }
            results.append(result)

            print(f"Result: K={k}, Conflicts={conflicts}, Time={execution_time:.2f}s, Valid={conflicts==0}")

    # Create DataFrame
    results_df = pd.DataFrame(results)

    # Display results
    print("\nComplete results:")
    print(results_df[['Graph', 'Algorithm', 'K', 'Known Best K', 'Conflicts', 'Time (s)', 'Valid Coloring']])

    return results_df

def verify_coloring(graph, colors):
    """
    Verify if a coloring is valid (no adjacent vertices have the same color).

    Args:
        graph: The graph
        colors: List of colors for each vertex

    Returns:
        Boolean indicating if the coloring is valid
    """
    for u in range(graph.n):
        for v in graph.get_neighbors(u):
            if colors[u] == colors[v]:
                return False
    return True

In [44]:

if __name__ == "__main__":
    import os
    import pandas as pd

    # Define the graph files to test
    graph_files = {
        'dsjc250.5' : '/content/dsjc250.5.col',
        'dsjc500.9':'/content/dsjc500.9.col',
        'dsjr500.1c':'/content/dsjr500.1c.col',
        'flat1000_76_0':'/content/flat1000_76_0.col',
        'r250.5':'/content/r250.5.col',
    }

    # Check if the graph files exist
    existing_files = {}
    for name, path in graph_files.items():
        if os.path.exists(path):
            existing_files[name] = path
        else:
            print(f"Warning: Graph file {path} not found. Skipping {name}.")

    if not existing_files:
        print("No graph files found. Creating a simple test graph...")
        # Create a simple test graph
        test_graph = Graph(5)
        test_graph.add_edge(0, 1)
        test_graph.add_edge(0, 2)
        test_graph.add_edge(1, 2)
        test_graph.add_edge(1, 3)
        test_graph.add_edge(2, 3)
        test_graph.add_edge(2, 4)
        test_graph.add_edge(3, 4)

        print("Testing algorithms on a simple graph:")
        compare_algorithms(test_graph)
    else:
        # Run tests on existing files
        results = test_coloring_algorithms(existing_files)

        # Save results to CSV
        results.to_csv('coloring_results.csv', index=False)
        print("Results saved to coloring_results.csv")

        # Create pivot table for better comparison
        pivot = pd.pivot_table(
            results,
            values=['K', 'Time (s)'],
            index=['Graph'],
            columns=['Algorithm']
        )

        print("\nPivot table of results:")
        print(pivot)


Processing graph: dsjc250.5
Graph loaded: 250 vertices
Testing algorithm: DSatur (Bucket)
Result: K=37, Conflicts=0, Time=0.05s, Valid=True
Testing algorithm: DSatur (Heap)
Result: K=37, Conflicts=0, Time=0.04s, Valid=True
Testing algorithm: Welsh-Powell
Result: K=41, Conflicts=0, Time=0.02s, Valid=True
Testing algorithm: RLF
Result: K=35, Conflicts=0, Time=0.13s, Valid=True

Processing graph: dsjc500.9
Graph loaded: 500 vertices
Testing algorithm: DSatur (Bucket)
Result: K=162, Conflicts=0, Time=0.40s, Valid=True
Testing algorithm: DSatur (Heap)
Result: K=170, Conflicts=0, Time=0.27s, Valid=True
Testing algorithm: Welsh-Powell
Result: K=169, Conflicts=0, Time=0.27s, Valid=True
Testing algorithm: RLF
Result: K=148, Conflicts=0, Time=1.91s, Valid=True

Processing graph: dsjr500.1c
Graph loaded: 500 vertices
Testing algorithm: DSatur (Bucket)
Result: K=91, Conflicts=0, Time=0.23s, Valid=True
Testing algorithm: DSatur (Heap)
Result: K=90, Conflicts=0, Time=0.19s, Valid=True
Testing algor

# Analysis of Graph Coloring Algorithm Performance

## Overview

This analysis evaluates the performance of different graph coloring algorithms on benchmark graphs from the DIMACS format. The key metrics measured include:

1. **Chromatic number (K)** - The minimum number of colors required
2. **Conflicts** - Number of adjacent vertices with the same color (0 for valid colorings)
3. **Execution Time** - Processing time in seconds
4. **Validity** - Whether all adjacent vertices have different colors

## Test Graphs

The following DIMACS benchmark graphs were used:

- `dsjc250.5` (250 vertices)
- `dsjc500.9` (500 vertices)
- `dsjr500.1c` (500 vertices)
- `flat1000_76_0` (1000 vertices)
- `r250.5` (250 vertices)

## Algorithms Tested

Four graph coloring algorithms were evaluated:

1. **DSatur (Bucket)** - DSatur algorithm using a bucket-based data structure
2. **DSatur (Heap)** - DSatur algorithm using a heap-based priority queue
3. **Welsh-Powell** - Greedy algorithm that colors vertices in descending order of degree
4. **Recursive Largest First (RLF)** - Recursive greedy algorithm selecting the largest independent sets

## Results by Graph

### dsjc250.5 (250 vertices)
- **RLF**: K=35 colors (best performer)
- **DSatur (Bucket)** and **DSatur (Heap)**: Both K=37 colors
- **Welsh-Powell**: K=41 colors (worst performer)
- All algorithms produced valid colorings with no conflicts
- **Welsh-Powell** was fastest (0.021s), **RLF** slowest (0.133s)

### dsjc500.9 (500 vertices)
- **RLF**: K=148 colors (best performer)
- **DSatur (Bucket)**: K=162 colors
- **Welsh-Powell**: K=169 colors
- **DSatur (Heap)**: K=170 colors (worst performer)
- **Welsh-Powell** and **DSatur (Heap)** were fastest (~0.27s)
- **RLF** was significantly slower (1.91s)

### dsjr500.1c (500 vertices)
- **DSatur (Heap)**: K=90 colors (best performer)
- **DSatur (Bucket)**: K=91 colors
- **RLF**: K=92 colors
- **Welsh-Powell**: K=100 colors (worst performer)
- **Welsh-Powell** was fastest (0.12s)
- **RLF** was slowest (1.10s)

### flat1000_76_0 (1000 vertices)
- **RLF**: K=104 colors (best performer)
- **DSatur (Bucket)**: K=114 colors
- **DSatur (Heap)**: K=115 colors
- **Welsh-Powell**: K=123 colors (worst performer)
- **DSatur (Heap)** was fastest among DSatur variants (0.43s)
- **RLF** was significantly slower (5.30s)

### r250.5 (250 vertices)
- **DSatur (Bucket)**: K=67 colors (best performer)
- **DSatur (Heap)**: K=68 colors
- **Welsh-Powell**: K=70 colors
- **RLF**: K=73 colors (worst performer)
- All algorithms were fast, with **Welsh-Powell** being quickest (0.03s)

## Performance Summary

### Colors Used (K) by Algorithm and Graph

| Graph          | DSatur (Bucket) | DSatur (Heap) | RLF   | Welsh-Powell |
|----------------|-----------------|---------------|-------|--------------|
| dsjc250.5      | 37              | 37            | 35    | 41           |
| dsjc500.9      | 162             | 170           | 148   | 169          |
| dsjr500.1c     | 91              | 90            | 92    | 100          |
| flat1000_76_0  | 114             | 115           | 104   | 123          |
| r250.5         | 67              | 68            | 73    | 70           |

### Execution Time (seconds) by Algorithm and Graph

| Graph          | DSatur (Bucket) | DSatur (Heap) | RLF     | Welsh-Powell |
|----------------|-----------------|---------------|---------|--------------|
| dsjc250.5      | 0.046           | 0.040         | 0.133   | 0.021        |
| dsjc500.9      | 0.401           | 0.268         | 1.913   | 0.271        |
| dsjr500.1c     | 0.234           | 0.187         | 1.101   | 0.118        |
| flat1000_76_0  | 0.610           | 0.433         | 5.297   | 0.761        |
| r250.5         | 0.038           | 0.040         | 0.178   | 0.032        |

## Key Findings

1. **Color Efficiency**:
   - **RLF** produced the best coloring results for 2 of 5 graphs (dsjc250.5, flat1000_76_0)
   - **DSatur variants** performed best for the other 3 graphs
   - **Welsh-Powell** consistently used more colors than other algorithms

2. **Time Efficiency**:
   - **Welsh-Powell** was generally the fastest algorithm
   - **RLF** was consistently the slowest, especially for larger graphs
   - **DSatur (Heap)** was faster than **DSatur (Bucket)** in most cases

3. **Algorithm Characteristics**:
   - **DSatur variants** offer good balance between color efficiency and speed
   - **Welsh-Powell** sacrifices color efficiency for speed
   - **RLF** produces good colorings but at significant time cost

## Conclusion

The optimal algorithm choice depends on specific requirements:

- For minimal colors regardless of time: **RLF** or **DSatur** variants
- For fastest results with acceptable coloring: **Welsh-Powell**
- For best balance of speed and quality: **DSatur (Heap)**
